**Imported Libraries:**

In [0]:
# Import the needed libraries
import numpy as np
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import GlobalAveragePooling2D, InputLayer, Dense, Reshape, Conv2D, MaxPooling2D, Input, TimeDistributed, GlobalMaxPooling1D, BatchNormalization, ZeroPadding2D, Dropout, Activation, Flatten, LSTM
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.utils import Sequence
from keras import optimizers
from keras.applications.vgg16 import VGG16
import glob
import os
import tensorflow as tf

Using TensorFlow backend.


In [0]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Unzip the dataset
!unzip -q 'drive/My Drive/MRNet-v1.0.zip'

**Helper Functions and Classes:**

Data generator class prepares the dataset for training or testing by doing the following:

1.   Load the training, validation or testing data of the given plane.
2.   Add 3 color channels to each case.
3.   Load the labels of the corresponding diagnosis. 
4.   Return a batch of size 1.




In [0]:
# Data generator class
# task = 0 - training, task = 1 - validation, task =  2 testing
class data_generator(Sequence):
  def __init__(self, directory, diagnosis, plane, task=0):
    self.directory = directory
    self.diagnosis = diagnosis
    self.plane = plane
    self.task = task
    self.labels = self.load_labels_from_dir()
    if self.task == 0 or self.task == 1:
      self.target_path = self.directory + 'train/{}/'.format(plane)
      self.target_labels = self.labels['train-{}'.format(diagnosis)]      
    elif self.task == 2:
      self.target_path = self.directory + 'valid/{}/'.format(plane)
      self.target_labels = self.labels['valid-{}'.format(diagnosis)]
    self.ids = list(self.target_labels['exam'])
    self.Y = list(self.target_labels['label'])
    if self.task == 0:
      self.ids = self.ids[:1017]
      self.Y = self.Y[:1017]
    elif self.task == 1:
      self.ids = self.ids[1017:]
      self.Y = self.Y[1017:]
    self.ids = map(lambda id: '0' * (4 - len(str(id))) + str(id), self.ids)
    self.paths = [self.target_path + id + '.npy' for id in self.ids]
  def __len__(self):
    return len(self.Y)
  
  def __getitem__(self, idx):
    batch_y = np.expand_dims(np.array(self.Y[idx]), 0)
    batch_x = np.load(self.paths[idx])
    batch_x = self.add_color_channels(batch_x)
    batch_x = np.expand_dims(batch_x, 0)
    return batch_x, batch_y

  def load_labels_from_dir(self):
    labels = {}
    for file in glob.glob(os.path.join(self.directory, '*.csv')):
      file_name = os.path.basename(file)
      labels[os.path.splitext(file_name)[0]] = pd.read_csv(file, index_col=False, names=['exam', 'label'])
    return labels
  
  def add_color_channels(self, im):
    return np.repeat(im[:, :, :, np.newaxis], 3, axis=3)

A function used to prepare the input data for the Logistic Regression layer.
This input is the result of the 3 planes' models predictions:

In [0]:
# input: models of 3 planes for a signle diagnosis, data generators of the 3 planes
# output: array containing the predictions of the 3 models
def predict_single_diagnosis(axial_model, sagittal_model, coronal_model, axial_data, sagittal_data, coronal_data):
  all_predictions = []
  for i in range (len(axial_data)):
    prediction_row = []
    x1, y1 = axial_data[i]
    prediction_row.append(axial_model.predict(x1)[0][0])
    x2, y2 = sagittal_data[i]
    prediction_row.append(sagittal_model.predict(x2)[0][0])
    x3, y3 = coronal_data[i]
    prediction_row.append(coronal_model.predict(x3)[0][0])
    all_predictions.append(prediction_row)
  
  return all_predictions

**Implemented Networks:**

1. AlexNet:

In [0]:
#AlexNet
def alexnet_from_scratch(input_layer):
  squeeze_layer = keras.layers.Lambda(lambda x: tf.keras.backend.squeeze(x, 0))(input_layer)
  normalization = BatchNormalization()(squeeze_layer)
  
  x = Conv2D(96, (11, 11), strides = (4, 4), activation = 'relu')(normalization)
  x = MaxPooling2D((3,3), strides=(2,2))(x)
  x = Conv2D(256, (5, 5), strides = (1, 1), padding = 'same', activation = 'relu')(x)
  x = MaxPooling2D((3,3), strides=(2,2))(x)
  x = Conv2D(384, (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(x)
  x = Conv2D(384, (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(x)
  x = Conv2D(256, (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(x)
  x = MaxPooling2D((3,3), strides=(2,2))(x)

  global_average_pool = GlobalAveragePooling2D()(x)
  global_average_pool = BatchNormalization()(global_average_pool)
  max_layer = keras.layers.Lambda(lambda x: tf.math.reduce_max(
      x, axis=0, keepdims=True, name=None
  ))(global_average_pool)
  max_layer = BatchNormalization()(max_layer)
  dense_layer = keras.layers.Dense(1, activation = 'sigmoid')(max_layer)
  dense_layer = Dropout(0.6)(dense_layer)

  model = keras.Model(input_layer, dense_layer)
  sgd = optimizers.SGD(lr=0.008, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics = ['accuracy'])
  return model

2. VGG:

In [0]:
  # VGG model from scratch
def vgg_from_scratch(input_layer):
  squeeze_layer = keras.layers.Lambda(lambda x: tf.keras.backend.squeeze(
      x, 0))(input_layer)
  normalization = BatchNormalization()(squeeze_layer)
  x = Conv2D(64, (3, 3), activation='relu')(normalization)
  x = Conv2D(64, (3, 3), activation='relu')(x)
  x = MaxPooling2D((2,2), strides=(2,2))(x)
  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPooling2D((2,2), strides=(2,2))(x)
  x = Conv2D(256, (3, 3), activation='relu')(x)
  x = Conv2D(256, (3, 3), activation='relu')(x)
  x = Conv2D(256, (3, 3), activation='relu')(x)
  x = MaxPooling2D((2,2), strides=(2,2))(x)
  x = Conv2D(512, (3, 3), activation='relu')(x)
  x = Conv2D(512, (3, 3), activation='relu')(x)
  x = Conv2D(512, (3, 3), activation='relu')(x)
  x = MaxPooling2D((2,2), strides=(2,2))(x)

  x = Conv2D(512, (3, 3), activation='relu')(x)
  x = Conv2D(512, (3, 3), activation='relu')(x)
  x = Conv2D(512, (3, 3), activation='relu')(x)
  x = MaxPooling2D((2,2), strides=(2,2))(x)

  global_average_pool = GlobalAveragePooling2D()(x)
  global_average_pool = BatchNormalization()(global_average_pool)
  max_layer = keras.layers.Lambda(lambda x: tf.math.reduce_max(
      x, axis=0, keepdims=True, name=None
  ))(global_average_pool)
  dense_layer = keras.layers.Dense(1, activation = 'sigmoid')(max_layer)
  dense_layer = Dropout(0.25)(dense_layer)
  model = keras.Model(input_layer, dense_layer)
  opt = optimizers.Adadelta(lr=0.001, decay=1e-6)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['accuracy'])
  return model

3. ResNet:

In [0]:
def resnet_from_scratch(input_layer):
  squeeze_layer = keras.layers.Lambda(lambda x: tf.keras.backend.squeeze(
      x, 0))(input_layer)
  normalization = BatchNormalization()(squeeze_layer)
  x = ZeroPadding2D((1,1))(normalization)
  x = Conv2D(64, 7, strides=(2, 2), activation="relu")(x)

  inp = MaxPooling2D((3, 3),padding='same', strides=(2, 2))(x)
  x = Conv2D(64, 3, padding='same', activation="relu")(inp)
  x = Conv2D(64, 3, padding='same', activation="linear")(x)
  inp = add([x, inp])
  inp = Activation('relu')(inp)
  x = Conv2D(64, 3, padding='same', activation="relu")(inp)
  x = Conv2D(64, 3, padding='same', activation="linear")(x)
  x = add([x, inp])
  inp = Activation('relu')(x)

  inp = MaxPooling2D((3, 3),padding='same', strides=(2, 2))(x)
  x = Conv2D(128, 3, padding='same', activation="relu")(inp)
  x = Conv2D(128, 3, padding='same', activation="linear")(x)
  inp = Conv2D(128, 3, padding='same', activation="relu")(inp)
  inp = add([x, inp])
  inp = Activation('relu')(inp)
  x = Conv2D(128, 3, padding='same', activation="relu")(inp)
  x = Conv2D(128, 3, padding='same', activation="linear")(x)
  x = add([x, inp])
  inp = Activation('relu')(x)

  inp = MaxPooling2D((3, 3),padding='same', strides=(2, 2))(x)
  x = Conv2D(256, 3, padding='same', activation="relu")(inp)
  x = Conv2D(256, 3, padding='same', activation="linear")(x)
  inp = Conv2D(256, 3, padding='same', activation="relu")(inp)
  inp = add([x, inp])
  inp = Activation('relu')(inp)
  x = Conv2D(256, 3, padding='same', activation="relu")(inp)
  x = Conv2D(256, 3, padding='same', activation="linear")(x)
  x = add([x, inp])
  inp = Activation('relu')(x)

  inp = MaxPooling2D((3, 3),padding='same', strides=(2, 2))(x)
  x = Conv2D(512, 3, padding='same', activation="relu")(inp)
  x = Conv2D(512, 3, padding='same', activation="linear")(x)
  inp = Conv2D(512, 3, padding='same', activation="relu")(inp)
  inp = add([x, inp])
  inp = Activation('relu')(inp)
  x = Conv2D(512, 3, padding='same', activation="relu")(inp)
  x = Conv2D(512, 3, padding='same', activation="linear")(x)
  x = add([x, inp])
  x = Activation('relu')(x)

  resnet = BatchNormalization()(x)

  global_average_pool = GlobalAveragePooling2D()(resnet)
  global_average_pool = BatchNormalization()(global_average_pool)
  max_layer = keras.layers.Lambda(lambda x: tf.math.reduce_max(
      x, axis=0, keepdims=True, name=None
  ))(global_average_pool)
  max_layer = BatchNormalization()(max_layer)
  dense_layer = keras.layers.Dense(1, activation = 'sigmoid')(max_layer)
  model = keras.Model(input_layer, dense_layer)
  sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics = ['accuracy'])
  return model

4. Inception:

In [0]:
def Conv(input_layer, f, k, p='same', s=(1,1)):
  layer = layers.Conv2D(filters=f, kernel_size=k, strides=s, padding=p, kernel_regularizer=l2(0.0))(input_layer)
  layer = layers.BatchNormalization(axis = 3)(layer)
  layer = layers.Activation('relu')(layer)
  return layer

In [0]:
def inception_from_scratch(inputLayer):
    layer = Conv(inputLayer, 32, (3,3),s=(2,2), p='valid')
    layer = Conv(layer, 32,(3,3) ,p = 'valid')
    layer = Conv(layer, 64, (3,3))
    layer = layers.MaxPooling2D((3,3), strides=(2,2))(layer)
    layer = Conv(layer, 80, (1, 1), p='valid')
    layer = Conv(layer, 192, (3, 3), p='valid')
    layer = layers.MaxPooling2D((3, 3), strides=(2, 2))(layer)
    
    for x in range(3):
        conv1 = Conv(layer, 64, (1, 1))
    
        conv3 = Conv(layer, 64, (1, 1))
        conv3 = Conv(conv3, 96, (3, 3))
        conv3 = Conv(conv3, 96, (3, 3))
    
        conv5 = Conv(layer, 48, (1, 1))
        conv5 = Conv(conv5, 64, (5, 5))
    
        Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(layer)
        Pool = Conv(Pool, 32, (1, 1))
        layer = layers.concatenate([conv1, conv3, conv5, Pool],axis=3)
    
    
    conv1 = Conv(layer, 384, (3, 3), s=(2, 2), p='valid')
    
    conv3 = Conv(layer, 64, (1, 1))
    conv3 = Conv(conv3, 96, (3, 3))
    conv3 = Conv(conv3, 96, (3, 3), s=(2, 2), p='valid')
    
    Pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(layer)
    layer = layers.concatenate([conv1, conv3, Pool], axis=3)
    
    conv1 = Conv(layer, 192, (1, 1))
    
    conv7 = Conv(layer, 128, (1, 1))
    conv7 = Conv(conv7, 128, (1, 7))
    conv7 = Conv(conv7, 192, (7, 1))
    
    conv7_2 = Conv(layer, 128, (1, 1))
    conv7_2 = Conv(conv7_2, 128, (7, 1))
    conv7_2 = Conv(conv7_2, 128, (1, 7))
    conv7_2 = Conv(conv7_2, 128, (7, 1))
    conv7_2 = Conv(conv7_2, 192, (1, 7))
    
    Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(layer)
    Pool = Conv(Pool, 192, (1, 1))
    
    
    for x in range(2):
      conv1 = Conv(layer, 192, (1, 1))
    
      conv7 = Conv(layer, 160, (1, 1))
      conv7 = Conv(conv7, 160, (1, 7))
      conv7 = Conv(conv7, 192, (7, 1))
    
      conv7_2 = Conv(layer, 160, (1, 1))
      conv7_2 = Conv(conv7_2, 160, (7, 1))
      conv7_2 = Conv(conv7_2, 160, (1, 7))
      conv7_2 = Conv(conv7_2, 160, (7, 1))
      conv7_2 = Conv(conv7_2, 192, (1, 7))
    
      Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(layer)
      Pool = Conv(Pool, 192, (1, 1))
    
      layer = layers.concatenate([conv1, conv7, conv7_2, Pool],axis=3)
    
    conv1 = Conv(layer, 192, (1, 1))
    
    conv7 = Conv(layer, 192, (1, 1))
    conv7 = Conv(conv7, 192, (1, 7))
    conv7 = Conv(conv7, 192, (7, 1))
    
    conv7_2 = Conv(layer, 192, (1, 1))
    conv7_2 = Conv(conv7_2, 192, (7, 1))
    conv7_2 = Conv(conv7_2, 192, (1, 7))
    conv7_2 = Conv(conv7_2, 192, (7, 1))
    conv7_2 = Conv(conv7_2, 192, (1, 7))
    
    Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(layer)
    Pool = Conv(Pool, 192, (1, 1))
    
    layer = layers.concatenate([conv1, conv7, conv7_2, Pool],axis=3)
    
    conv1 = Conv(layer, 192, (1, 1))
    conv1 = Conv(layer, 320, (3,3),s=(2, 2),p='valid')
    
    conv7_2 = Conv(layer, 192, (1, 1))
    conv7_2 = Conv(conv7_2, 192, (1, 7))
    conv7_2 = Conv(conv7_2, 192, (7, 1))
    conv7_2 = Conv(conv7_2, 192,(3,3),s=(2,2),p='valid')
    
    Pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(layer)
    layer = layers.concatenate([conv1,conv7_2, Pool],axis=3)
    
    for i in range(2):
        conv1 = Conv(layer, 320, (1, 1))
    
        conv3 = Conv(layer, 384, (1, 1))
        conv3_11 = Conv(conv3, 384, (1, 3))
        conv3_12 = Conv(conv3, 384, (3, 1))
        conv3 = layers.concatenate([conv3_11,conv3_12],axis=3)
    
        conv32 = Conv(layer, 448, (1, 1))
        conv32 = Conv(conv32, 384, (3, 3))
        conv3_21 = Conv(conv32, 384, (1, 3))
        conv3_22 = Conv(conv32, 384, (3, 1))
        conv32 = layers.concatenate([conv3_21,conv3_22],axis=3)
    
        Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(layer)
        Pool = Conv(Pool, 192, (1, 1))
        layer = layers.concatenate([conv1,conv3,conv32, Pool],axis=3)
    return layer

**Transfer Learning of VGG16**

In [0]:
def transfer_learning_with_vgg(input_layer):
  squeeze_layer = keras.layers.Lambda(lambda x: tf.keras.backend.squeeze(
      x, 0))(input_layer)
  normalization = BatchNormalization()(squeeze_layer)
  vgg = VGG16(weights='imagenet',include_top=False)
  for layer in vgg.layers:
    layer.trainable = False
  vgg = vgg(normalization)
  global_average_pool = GlobalAveragePooling2D()(vgg)
  global_average_pool = BatchNormalization()(global_average_pool)
  max_layer = keras.layers.Lambda(lambda x: tf.math.reduce_max(
      x, axis=0, keepdims=True, name=None
  ))(global_average_pool)
  dense_layer = keras.layers.Dense(1, activation = 'sigmoid')(max_layer)
  dense_layer = Dropout(0.1)(dense_layer)
  model = keras.Model(input_layer, dense_layer)
  opt = optimizers.Adadelta(lr=0.01)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['accuracy'])
  model.layers[3].trainable = False
  return model

**Training Phase:**


Training steps for a single model (ex: meniscus, coronal) using a specific network:

In [0]:
input_layer = keras.Input(shape=(None,256,256,3))
model = transfer_learning_with_vgg(input_layer)
model.summary()

58892288/58889256 [==============================] - 2s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 256, 256, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 3)       12        
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
____________________________________________________________

In [0]:
train_generator = data_generator('MRNet-v1.0/', 'meniscus', 'coronal', task=0)
valid_generator = data_generator('MRNet-v1.0/', 'meniscus', 'coronal', task=1)

In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=1017, epochs=10, validation_data=valid_generator, validation_steps=113)

In [0]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,7)
plt.show()

In [0]:
model.save('drive/My Drive/vgg_coronal_meniscus.h5')

# This notebook is a collective summary for the training that each one has done on his own notebooks to parallelize the training phase among us. So, in this part we only load the models that we trained previously

**Logistic Regression Layer:**

1. Loading the models of 3 planes:


In [0]:
axial_model = keras.models.load_model('drive/My Drive/vgg_axial_meniscus_tl.h5', custom_objects={'tf': tf})
sagittal_model = keras.models.load_model('drive/My Drive/vgg_sagittal_meniscus_tl.h5', custom_objects={'tf': tf})
coronal_model = keras.models.load_model('drive/My Drive/vgg_coronal_meniscus_tl.h5', custom_objects={'tf': tf})

2. Loading training data generator:


In [0]:
t_axial_data_generator = data_generator('MRNet-v1.0/', 'meniscus', 'axial', task = 0)
t_sagittal_data_generator = data_generator('MRNet-v1.0/', 'meniscus', 'sagittal', task = 0)
t_coronal_data_generator = data_generator('MRNet-v1.0/', 'meniscus', 'coronal', task = 0)

3. Loading validation data generator:

In [0]:
v_axial_data_generator = data_generator('MRNet-v1.0/', 'meniscus', 'axial', task = 1)
v_sagittal_data_generator = data_generator('MRNet-v1.0/', 'meniscus', 'sagittal', task = 1)
v_coronal_data_generator = data_generator('MRNet-v1.0/', 'meniscus', 'coronal', task = 1)

4. Preparing the training and validation data for the LR layer (predictions of the 3 models)

In [0]:
train_all_predictions = predict_single_diagnosis(axial_model, sagittal_model, coronal_model, 
                                                t_axial_data_generator, t_sagittal_data_generator, t_coronal_data_generator)

valid_all_predictions = predict_single_diagnosis(axial_model, sagittal_model, coronal_model, 
                                                v_axial_data_generator, v_sagittal_data_generator, v_coronal_data_generator)

5. Fully connected (dense) layer:

In [0]:
dense_layer = Sequential([Dense(1, input_shape = (3, ), activation = 'sigmoid')])
sgd = optimizers.SGD(lr=0.1)
dense_layer.compile(optimizer=sgd, loss='binary_crossentropy', metrics = ['accuracy'])
dense_layer.summary()

6. Training LR layer:

In [0]:
all_labels = load_labels_from_dir('MRNet-v1.0')
train_labels = list(all_labels['train-meniscus']['label'])

valid_labels = train_labels[1017:]
train_labels = train_labels[:1017]

validation = (np.array(valid_all_predictions), valid_labels)
history = dense_layer.fit(np.array(train_all_predictions), train_labels, steps_per_epoch=1017, epochs=100, validation_data= validation, validation_steps= 113 )

In [0]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

**Testing Phase:**

In [0]:
# Loading testing data generator
axial_test_generator = data_generator('MRNet-v1.0/', 'meniscus', 'axial', task = 2)
sagittal_test_generator = data_generator('MRNet-v1.0/', 'meniscus', 'sagittal', task = 2)
coronal_test_generator = data_generator('MRNet-v1.0/', 'meniscus', 'coronal' , task = 2)

In [0]:
# Test Input
test_all_predictions = predict_single_diagnosis(axial_model, sagittal_model, coronal_model, 
                                                axial_test_generator, sagittal_test_generator, coronal_test_generator)

In [0]:
# Test Output
all_labels = load_labels_from_dir('MRNet-v1.0')
test_all_labels = list(all_labels['valid-meniscus']['label'])

In [0]:
# Evaluation
dense_layer.evaluate(np.array(test_all_predictions), test_all_labels)